In [1]:
%load_ext autoreload
%autoreload 2
import sys, warnings, time, numpy, yaml
sys.path.append("../src/") # go to parent dir
from data_access import get_X, get_y, get_train_test
# from models import get_model_benchmark1, get_model_benchmark2, show_importance
from models.model import Model
from models.factory import ModelFactory
from models.benchmark import Benchmark1, Benchmark2
warnings.filterwarnings('ignore')

In [2]:
train_data = get_X('train')
train_scores = get_y()
test_data = get_X('test')
X_train, y_train, X_test, y_test, X_valid, y_valid, target = get_train_test(train_size=0.8, random_state=42)

In [3]:
%load_ext autoreload
%autoreload 2
benchmark1 = Benchmark1(X_train, y_train, X_valid, y_valid, train_scores)
benchmark1.train()
benchmark1.evaluate(X_test)
# benchmark1.save(test_data)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


0.4401

In [4]:
%load_ext autoreload
%autoreload 2
benchmark2 = Benchmark2(X_train, y_train, X_valid, y_valid, train_scores)
benchmark2.train()
benchmark2.evaluate(X_test)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


0.4742

In [5]:
benchmark2.save(test_data)

submission=          ID  HOME_WINS  DRAW  AWAY_WINS
0      12303          1     0          0
1      12304          0     0          1
2      12305          1     0          0
3      12306          1     0          0
4      12307          1     0          0
...      ...        ...   ...        ...
25363  37666          1     0          0
25364  37667          1     0          0
25365  37668          1     0          0
25366  37669          1     0          0
25367  37670          1     0          0

[25368 rows x 4 columns]


In [6]:
params_by_name={
    'benchmark2': {
          'booster': 'gbtree',
          'tree_method':'hist',
          'max_depth': 8, 
          'learning_rate': 0.025,
          'objective': 'multi:softprob',
          'num_class': 2,
          'eval_metric':'mlogloss'
        },
    'benchmark3': {
          'booster': 'gblinear',
          'tree_method':'hist',
          'max_depth': 8, 
          'learning_rate': 0.025,
          'objective': 'multi:softprob',
          'num_class': 2,
          'eval_metric':'mlogloss'
        },
}
for name, params in params_by_name.items():
    benchmark2 = Benchmark2(X_train, y_train, X_valid, y_valid, train_scores, params)
    benchmark2.train()
    benchmark2.evaluate(X_test)
    benchmark2.name = name
    benchmark2.save(test_data)

submission=          ID  HOME_WINS  DRAW  AWAY_WINS
0      12303          1     0          0
1      12304          0     0          1
2      12305          1     0          0
3      12306          1     0          0
4      12307          1     0          0
...      ...        ...   ...        ...
25363  37666          1     0          0
25364  37667          1     0          0
25365  37668          1     0          0
25366  37669          1     0          0
25367  37670          1     0          0

[25368 rows x 4 columns]
submission=          ID  HOME_WINS  DRAW  AWAY_WINS
0      12303          1     0          0
1      12304          1     0          0
2      12305          1     0          0
3      12306          1     0          0
4      12307          1     0          0
...      ...        ...   ...        ...
25363  37666          1     0          0
25364  37667          1     0          0
25365  37668          1     0          0
25366  37669          1     0          0
25367  37

In [7]:
%load_ext autoreload
%autoreload 2
def get_params(booster, tree_method, eval_metric='mlogloss'):
    return {
          'booster': booster,
          'tree_method': tree_method,
          'max_depth': 8, 
          'learning_rate': 0.025,
          'objective': 'multi:softprob',
          'num_class': 2,
          'eval_metric':eval_metric
        }
# boosters = ['gbtree', 'gblinear', 'dart']
boosters = ['gblinear']
# tree_methods = ['auto', 'exact', 'approx', 'hist']
tree_methods = ['hist']
# eval_metrics = ['mphe', 'merror', 'mlogloss', 'auc']
eval_metrics = ['mlogloss']
for booster in boosters:
    for tree_method in tree_methods:
        for eval_metric in eval_metrics:
          start = time.time()
          name = f'{booster}_{tree_method}_{eval_metric}'
          params = get_params(booster, tree_method, eval_metric)
          benchmark2 = Benchmark2(X_train, y_train, X_valid, y_valid, train_scores, params)
          benchmark2.train()
          score = benchmark2.evaluate(X_test)
          end = time.time()
          print(f'{name}={score} in {numpy.round((end-start), 2)}s')

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
gblinear_hist_mlogloss=0.4758 in 3.97s


In [13]:
name = 'benchmark3'
with open('../confs/models.yaml', 'r') as file:
    configurations = yaml.safe_load(file)
factory = ModelFactory(configurations, X_train, y_train, X_valid, y_valid, train_scores)
model = factory.get_model(name) 
model.train()
print(model.evaluate(X_test))

0.4754


In [15]:
%load_ext autoreload
%autoreload 2
with open('../confs/models.yaml', 'r') as file:
    configurations = yaml.safe_load(file)
factory = ModelFactory(configurations, X_train, y_train, X_valid, y_valid, train_scores)
for model in factory.get_models():
    start = time.time()
    model.train()
    score = model.evaluate(X_test)
    end = time.time()
    print(f'{name}={score} in {numpy.round((end-start), 2)}s')

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
benchmark3=0.4742 in 13.71s
benchmark3=0.477 in 3.67s
